In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import os

import data
from model import Model

import colours as c

dir = os.getcwd()
dir_main = os.path.dirname(os.path.abspath(dir))

In [ ]:
d = data.Data()
d = data.Scorer()
m = Model(d.smiles_vocabulary)

In [ ]:
with tf.Session() as session:
    
    labels = None
    
    saver = tf.train.Saver()
    saver.restore(session, dir_main + "/trained/Solanaceae-E")
    
    print("Model restored.")
    
    # Manually evaluate compound
    compound_smiles = "CN1C=CC2=CC(NC(=O)NC3=CC(C)=NS3)=CC=C12"
    
    feed_smiles = [d.indexSmiles(compound_smiles, padding=200)]
    
    predicted = session.run([m.output], feed_dict={m.batch_size: 1, m.input: feed_smiles})
    rounded = np.round(predicted[0][0])
    
    print(c.BLUE + c.BOLD + "\nPredicted values:" + c.END)
    print("Logistics: {}".format(predicted[0][0]))
    print("Rounded: {}".format(rounded))
    print("String labels: {}".format(d.labelsToReadable(rounded)))
    
    if compound_smiles in d.compounds:
        string_labels = d.compounds[compound_smiles]["labels"]
        labels = [d.classes.index(label) for label in string_labels]
        
        print(c.RED + c.BOLD + "\nLiterature database:" + c.END)
        print("Expect: {}".format(d.createLabels(labels)))
        print("Tags: {}".format(string_labels))
        
    else:
        print(c.RED + c.BOLD + "\nCompound not in database." + c.END)
    
    print(c.BOLD + "\nAvailable classes:" + c.END)
    print(d.classes)
    
    figure = plt.figure(figsize=(20, 5))
    plt.bar(d.classes, predicted[0][0], color="blue")
    plt.ylim([0, 1])
    
    if labels != None:
        figure = plt.figure(figsize=(20, 5))
        plt.bar(d.classes, d.createLabels(labels), color="red")
        plt.ylim([0, 1])